In [0]:
import pyspark
from pyspark.sql import DataFrameWriter
import pandas as pd
from pyspark.sql.functions import *
import seaborn as sns 
import matplotlib.pyplot as plt
import sklearn
import datetime
import numpy as np

In [0]:
spark.conf.set("fs.azure.account.key.carehealthcareblobsa.dfs.core.windows.net",
    dbutils.secrets.get(scope="healthcarescope", key="blobaccesskey"))

In [0]:
patient_data = spark.read.csv("abfss://rawhealthcaredata@carehealthcareblobsa.dfs.core.windows.net/Patient_records.csv", header=True, inferSchema=True)

In [0]:
#patient_data.show(10,False)

In [0]:
#patient_data.printSchema()

In [0]:
patent_trans_data = patient_data.select("Patient_id","Patient_name","patient_gender","patient_birth_date","disease_name","city","hospital_id").distinct()

In [0]:
#patient_data.select([count(when(col(c).isNull(), c)).alias(c) for c in patient_data.columns]).show()

In [0]:
#patient_data.groupby(patient_data.columns).count().where("count > 1").show() 

In [0]:
patient_data = patient_data.dropDuplicates()

In [0]:
#patent_trans_data.show(5)

In [0]:
patent_trans_data = patent_trans_data.fillna({"Patient_name":"Guest/NA"})

In [0]:
#patent_trans_data.show(5,False)

In [0]:
patent_trans_data = patent_trans_data.withColumn("Patient_age", (months_between(current_date(), col('patient_birth_date'))/12).cast("int"))

In [0]:
#atent_trans_data.show(6,False)

In [0]:
patent_trans_data = patent_trans_data.drop(col("patient_birth_date"))

In [0]:
#patent_trans_data.show(5,False)

In [0]:
#patent_trans_data.select("*").filter(col("patient_id")==121783).show(5)

In [0]:
output_container_path = "abfss://stagginghealthcaredata@carehealthcareblobsa.dfs.core.windows.net"
output_blob_folder = "stagingdatasets/" 
patent_trans_data.coalesce(1).write.mode("overwrite").option("header", "true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path, "%s/patientdatastagging.csv" % output_container_path)